In [1]:
import os
import pandas as pd

root_folder = "/home/trungdc/unimelb/MAST30024/asm/mast30034_2021_s2_project_1-alexdang02-1"
data_dir = os.path.join(root_folder, "Data")
SQLOutput_dir = os.path.join(root_folder, "code/SparkSQL_Output")
plot_dir = os.path.join(root_folder, "Plots")
resource_save = os.path.join(root_folder,"code", "EDA Visualisation", "stat")

In [2]:
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [4]:
fhvcompany = ["HV0002", "HV0003", "HV0004", "HV0005"]
juno = "B02914 B02907 B02907 B02907".split(" ")
lyft = "B02510 B02510".split(" ") 
uber = "B02877 B02866 B02882 B02869 B02617 B02876 B02865 B02512 B02888 B02864 B02883 B02875 B02682 B02880 B02870 B02404 B02598 B02765 B02879 B02867 B02878 B02887 B02872 B02836 B02884 B02835 B02764 B02889 B02871 B02395 B03136 B02800".split(" ")
base_numbers = []
base_numbers = juno + lyft + uber

In [5]:
def convert_base_to_company(Dispatching_base_number):
    if Dispatching_base_number in lyft:
        return "lyft"
    elif Dispatching_base_number in juno:
        return "juno"
    else:
        return "uber"
udfconvert_base_to_company = udf(convert_base_to_company, StringType())

def convert_license_to_company(hvfhs_license_num):
    if hvfhs_license_num == "HV0002":
        return "juno"
    elif hvfhs_license_num == "HV0003":
        return "uber"
    elif hvfhs_license_num == "HV0004":
        return "via"
    else:
        return "lyft"
udfconvert_license_to_company = udf(convert_license_to_company, StringType())

# fvfhv data

## 2018 data

In [33]:
fhv2018_count = []
month = 1
year = 2018
for file in sorted(os.listdir(os.path.join(data_dir,"Trip","fhvhv", "2018"))):
    if file.endswith(".csv"):
        file_path = os.path.join(data_dir,"Trip","fhvhv", "2018", file)
        fhv2018 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file_path) 
        fhv2018 = fhv2018.filter(fhv2018.Dispatching_base_number.isin(base_numbers)) \
        .withColumn("Company", udfconvert_base_to_company(fhv2018.Dispatching_base_number)) 
        fhv2018.createOrReplaceTempView("fhv2018")
        SQL = """
        SELECT Company, COUNT(*) AS Frequency
        FROM fhv2018
        GROUP BY Company
        """
        output = spark.sql(SQL).toPandas()
        output["Month"] = [month] * len(output.index)
        output["Year"] = [year] * len(output.index)
        print(output)
        output.to_pickle(os.path.join(resource_save, "Company",f"{month}-{year}.pkl" ))
        count = fhv2018.count()
        print(f"File {file} has {count} rows")
        fhv2018_count.append(count)
        month += 1
pd.DataFrame(fhv2018_count).to_pickle(os.path.join(resource_save, "fhv2018.pkl"))


  Company  Frequency  Month  Year
0    lyft    3141012      1  2018
1    juno    1378191      1  2018
2    uber   12688559      1  2018
File fhv_tripdata_2018-01.csv has 17207762 rows
  Company  Frequency  Month  Year
0    lyft    3144050      2  2018
1    juno    1329836      2  2018
2    uber   12434761      2  2018
File fhv_tripdata_2018-02.csv has 16908647 rows
  Company  Frequency  Month  Year
0    lyft    3471602      3  2018
1    juno    1402393      3  2018
2    uber   14489383      3  2018
File fhv_tripdata_2018-03.csv has 19363378 rows
  Company  Frequency  Month  Year
0    lyft    3332985      4  2018
1    juno     999872      4  2018
2    uber   14225839      4  2018
File fhv_tripdata_2018-04.csv has 18558696 rows
  Company  Frequency  Month  Year
0    lyft    3400356      5  2018
1    juno    1244326      5  2018
2    uber   14591260      5  2018
File fhv_tripdata_2018-05.csv has 19235942 rows
  Company  Frequency  Month  Year
0    lyft    3664808      6  2018
1    juno   

## 2019 data

In [10]:
fhv2019_count = []
month = 1
year = 2019
for file in sorted(os.listdir(os.path.join(data_dir,"Trip","fhvhv", "2019"))):
    if file.endswith(".csv"):
        file_path = os.path.join(data_dir,"Trip","fhvhv", "2019", file)
        fhv2019 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file_path) 
        if file == "fhv_tripdata_2019-01.csv":
            fhv2019 = fhv2019.filter(fhv2019.dispatching_base_num.isin(base_numbers)) \
            .withColumn("Company", udfconvert_base_to_company(fhv2019.dispatching_base_num)) 
        else:
            fhv2019 = fhv2019.withColumn("Company", udfconvert_license_to_company(fhv2019.hvfhs_license_num))
        fhv2019.createOrReplaceTempView("fhv2019")
        SQL = """
        SELECT Company, COUNT(*) AS Frequency
        FROM fhv2019
        GROUP BY Company
        """
        output = spark.sql(SQL).toPandas()
        output["Month"] = [month] * len(output.index)
        output["Year"] = [year] * len(output.index)
        print(output)
        output.to_pickle(os.path.join(resource_save, "Company",f"{month}-{year}.pkl" ))
        count = fhv2019.count()
        print(f"File {file} has {count} rows")
        fhv2019_count.append(count)
        month += 1
pd.DataFrame(fhv2019_count).to_pickle(os.path.join(resource_save, "fhv2019.pkl"))

  Company  Frequency  Month  Year
0    lyft    4623412      1  2019
1    juno     733671      1  2019
2    uber   15332400      1  2019
File fhv_tripdata_2019-01.csv has 20689483 rows
  Company  Frequency  Month  Year
0    lyft    4690916      2  2019
1    juno     979266      2  2019
2    uber   13504994      2  2019
3     via     983926      2  2019
File fhvhv_tripdata_2019-02.csv has 20159102 rows
  Company  Frequency  Month  Year
0    lyft    4699040      3  2019
1    juno     835215      3  2019
2    uber   17248340      3  2019
3     via    1082003      3  2019
File fhvhv_tripdata_2019-03.csv has 23864598 rows
  Company  Frequency  Month  Year
0    lyft    4290694      4  2019
1    juno     740532      4  2019
2    uber   15657399      4  2019
3     via    1046142      4  2019
File fhvhv_tripdata_2019-04.csv has 21734767 rows
  Company  Frequency  Month  Year
0    lyft    4681802      5  2019
1    juno     742053      5  2019
2    uber   15876033      5  2019
3     via    1029266

## 2020 data

In [40]:
fhv2019_count = []
month = 1
year = 2020
for file in sorted(os.listdir(os.path.join(data_dir,"Trip","fhvhv", "2020"))):
    if file.endswith(".csv"):
        file_path = os.path.join(data_dir,"Trip","fhvhv", "2020", file)
        fhv2019 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file_path) 
        fhv2019 = fhv2019.filter(fhv2019.dispatching_base_num.isin(base_numbers)) \
        .withColumn("Company", udfconvert_license_to_company(fhv2019.hvfhs_license_num))

        fhv2019.createOrReplaceTempView("fhv2019")
        SQL = """
        SELECT Company, COUNT(*) AS Frequency
        FROM fhv2019
        GROUP BY Company
        """
        output = spark.sql(SQL).toPandas()
        output["Month"] = [month] * len(output.index)
        output["Year"] = [year] * len(output.index)
        print(output)
        output.to_pickle(os.path.join(resource_save, "Company",f"{month}-{year}.pkl" ))
         

        count = fhv2019.count()
        print(f"File {file} has {count} rows")
        fhv2019_count.append(count)
        month += 1
pd.DataFrame(fhv2019_count).to_pickle(os.path.join(resource_save, "fhv2020.pkl"))

  Company  Frequency  Month  Year
0    lyft    5272933      1  2020
1    uber   14582477      1  2020
2     via     712600      1  2020
File fhvhv_tripdata_2020-01.csv has 20568010 rows
  Company  Frequency  Month  Year
0    lyft    5335932      2  2020
1    uber   15743375      2  2020
2     via     641941      2  2020
File fhvhv_tripdata_2020-02.csv has 21721248 rows
  Company  Frequency  Month  Year
0    lyft    3216493      3  2020
1    uber    9836763      3  2020
2     via     336606      3  2020
File fhvhv_tripdata_2020-03.csv has 13389862 rows
  Company  Frequency  Month  Year
0    lyft    1142917      4  2020
1    uber    3102835      4  2020
2     via      65991      4  2020
File fhvhv_tripdata_2020-04.csv has 4311743 rows
  Company  Frequency  Month  Year
0    lyft    1619967      5  2020
1    uber    4359273      5  2020
2     via     109127      5  2020
File fhvhv_tripdata_2020-05.csv has 6088367 rows
  Company  Frequency  Month  Year
0    lyft    2307960      6  2020
1   

In [17]:
fhv2019 = fhv2019.withColumn("month", month(fhv2019.pickup_datetime)) \
    .withColumn("pickup_datetime", to_timestamp(fhv2019.pickup_datetime, 'yyyy-MM-dd HH:mm:ss') ) \
.withColumn("dropoff_datetime", to_timestamp(fhv2019.dropoff_datetime, 'yyyy-MM-dd HH:mm:ss') ) 
.withColumn("year", year(fhv2019.pickup_datetime)) \
.withColumn("Company", udfconvert_base_to_company(fhv2019.dispatching_base_num)) \
.withColumn("duration", round((fhv2019.dropoff_datetime.cast("long")  - fhv2019.pickup_datetime.cast("long"))/60)) \
.drop("SR_Flag",'Dispatching_base_number', "Dispatching_base_num", "DropOff_datetime","Pickup_DateTime"  ) 
fhv2019.createOrReplaceTempView("fhv2019")
fhv2019.show()

+------------+------------+-----+----+-------+--------+
|PULocationID|DOLocationID|month|year|Company|duration|
+------------+------------+-----+----+-------+--------+
|          90|          48|    1|2019|   uber|    33.0|
|          48|         144|    1|2019|   uber|    26.0|
|          48|         144|    1|2019|   uber|    25.0|
|         112|         112|    1|2019|   uber|     7.0|
|         112|         145|    1|2019|   uber|    10.0|
|         145|         225|    1|2019|   uber|    23.0|
|         181|          54|    1|2019|   uber|    14.0|
|         181|         181|    1|2019|   uber|     4.0|
|          18|          78|    1|2019|   uber|    14.0|
|         127|         244|    1|2019|   uber|     9.0|
|         243|         243|    1|2019|   uber|     7.0|
|         144|          79|    1|2019|   uber|     8.0|
|         224|         263|    1|2019|   uber|    19.0|
|         263|          41|    1|2019|   uber|    11.0|
|         239|         265|    1|2019|   uber|  

# Green data

In [50]:
green_count = []
for file in sorted(os.listdir(os.path.join(data_dir,"Trip","green"))):
    if file.endswith(".csv"):
        file_path = os.path.join(data_dir,"Trip","green", file)
        fhv2019 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file_path)
        count = fhv2019.count()
        green_count.append(count)
        print(f"File {file} has {count} rows")
pd.DataFrame(green_count).to_pickle(os.path.join(resource_save, "green.pkl"))

File green_tripdata_2018-01.csv has 793529 rows
File green_tripdata_2018-02.csv has 769940 rows
File green_tripdata_2018-03.csv has 837149 rows
File green_tripdata_2018-04.csv has 800084 rows
File green_tripdata_2018-05.csv has 797233 rows
File green_tripdata_2018-06.csv has 739373 rows
File green_tripdata_2018-07.csv has 684455 rows
File green_tripdata_2018-08.csv has 666376 rows
File green_tripdata_2018-09.csv has 666708 rows
File green_tripdata_2018-10.csv has 710510 rows
File green_tripdata_2018-11.csv has 656573 rows
File green_tripdata_2018-12.csv has 685373 rows
File green_tripdata_2019-01.csv has 630918 rows
File green_tripdata_2019-02.csv has 575685 rows
File green_tripdata_2019-03.csv has 601102 rows
File green_tripdata_2019-04.csv has 514392 rows
File green_tripdata_2019-05.csv has 504887 rows
File green_tripdata_2019-06.csv has 471052 rows
File green_tripdata_2019-07.csv has 470743 rows
File green_tripdata_2019-08.csv has 449695 rows
File green_tripdata_2019-09.csv has 4490

# Yellow data

In [48]:
green_count = []
for file in sorted(os.listdir(os.path.join(data_dir,"Trip","yellow", "2018"))):
    if file.endswith(".csv"):
        file_path = os.path.join(data_dir,"Trip","yellow", "2018", file)
        fhv2019 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file_path)
        count = fhv2019.count()
        green_count.append(count)
        print(f"File {file} has {count} rows")
pd.DataFrame(green_count).to_pickle(os.path.join(resource_save, "yellow2018.pkl"))

File yellow_tripdata_2018-01.csv has 8759874 rows
File yellow_tripdata_2018-02.csv has 8492076 rows
File yellow_tripdata_2018-03.csv has 9430376 rows
File yellow_tripdata_2018-04.csv has 9305515 rows
File yellow_tripdata_2018-05.csv has 9224063 rows
File yellow_tripdata_2018-06.csv has 8713831 rows
File yellow_tripdata_2018-07.csv has 7849748 rows
File yellow_tripdata_2018-08.csv has 7849134 rows
File yellow_tripdata_2018-09.csv has 8040133 rows
File yellow_tripdata_2018-10.csv has 8821105 rows
File yellow_tripdata_2018-11.csv has 8145164 rows
File yellow_tripdata_2018-12.csv has 8173231 rows


In [43]:
green_count = []
for file in sorted(os.listdir(os.path.join(data_dir,"Trip","yellow", "2019"))):
    if file.endswith(".csv"):
        file_path = os.path.join(data_dir,"Trip","yellow", "2019", file)
        fhv2019 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file_path)
        count = fhv2019.count()
        green_count.append(count)
        print(f"File {file} has {count} rows")
pd.DataFrame(green_count).to_pickle(os.path.join(resource_save, "yellow2019.pkl"))

File yellow_tripdata_2019-01.csv has 7667792 rows
File yellow_tripdata_2019-02.csv has 7019375 rows
File yellow_tripdata_2019-03.csv has 7832545 rows
File yellow_tripdata_2019-04.csv has 7433139 rows
File yellow_tripdata_2019-05.csv has 7565261 rows
File yellow_tripdata_2019-06.csv has 6941024 rows
File yellow_tripdata_2019-07.csv has 6310419 rows
File yellow_tripdata_2019-08.csv has 6073357 rows
File yellow_tripdata_2019-09.csv has 6567788 rows
File yellow_tripdata_2019-10.csv has 7213891 rows
File yellow_tripdata_2019-11.csv has 6878111 rows
File yellow_tripdata_2019-12.csv has 6896317 rows


In [11]:
green_count = []
for file in sorted(os.listdir(os.path.join(data_dir,"Trip","yellow", "2020"))):
    if file.endswith(".csv"):
        file_path = os.path.join(data_dir,"Trip","yellow", "2020", file)
        fhv2019 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file_path)
        count = fhv2019.count()
        green_count.append(count)
        print(f"File {file} has {count} rows")
pd.DataFrame(green_count).to_pickle(os.path.join(resource_save, "yellow2020.pkl"))

File yellow_tripdata_2020-01.csv has 6405008 rows
File yellow_tripdata_2020-02.csv has 6299354 rows
File yellow_tripdata_2020-03.csv has 3007292 rows
File yellow_tripdata_2020-04.csv has 237993 rows
File yellow_tripdata_2020-05.csv has 348371 rows
File yellow_tripdata_2020-06.csv has 549760 rows
File yellow_tripdata_2020-07.csv has 800412 rows
File yellow_tripdata_2020-08.csv has 1007284 rows
File yellow_tripdata_2020-09.csv has 1341012 rows
File yellow_tripdata_2020-10.csv has 1681131 rows
File yellow_tripdata_2020-11.csv has 1508985 rows
File yellow_tripdata_2020-12.csv has 1461897 rows
